# Hidden Markov Models

Hidden Markov Models is a type of a random process. We begin with a few “states” for the chain, {S₁,…,Sₖ}; For instance, if our chain represents the cricket match, we can have {Boundary, single run, wicket}. Markov Chains are often described by a graph with transition probabilities, i.e, the probability of moving to state j from state i, which are denoted by pᵢ,ⱼ. Let us generate a sequence of one over, in each 1 denotes the Batsman is batting and 0 denotes Bowler is batting. We will use the algorithm to find the most likely event occurance for the over ( 6 balls).

In [31]:
import numpy as np
import pandas as pd

obs_map = {'Batsman':0, 'Bowler':1}
obs = np.array([1,1,0,1,0,0])

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

print("Simulated Observations:\n",pd.DataFrame(np.column_stack([obs, obs_seq]),columns=['Obs_code', 'Obs_seq']) )



Simulated Observations:
   Obs_code  Obs_seq
0        1   Bowler
1        1   Bowler
2        0  Batsman
3        1   Bowler
4        0  Batsman
5        0  Batsman


In [50]:
pi = [0.6,0.4] # initial probabilities vector
states = ['Batsman', 'Bowler']
hidden_states = ['Boundary', 'single run', 'wicket']
pi = [0, 0.2, 0.8]
state_space = pd.Series(pi, index=hidden_states, name='states')
a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.3, 0.5, 0.2]
a_df.loc[hidden_states[1]] = [0.6, 0.2, 0.2]
a_df.loc[hidden_states[2]] = [0.2, 0.7, 0.1]
print("\n HMM matrix:\n", a_df)
a = a_df.values




 HMM matrix:
            Boundary single run wicket
Boundary        0.3        0.5    0.2
single run      0.6        0.2    0.2
wicket          0.2        0.7    0.1


Suppose we have the Markov Chain from above, with three states (Boundary, single run and wicket), P - the transition probability matrix and q — the initial probabilities. This is the invisible Markov Chain — suppose we know who will be facing (Batsman or Bowler) the ball.

In [51]:
observable_states = states
b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.8,0.2]
b_df.loc[hidden_states[1]] = [0.6,0.4]
b_df.loc[hidden_states[2]] = [0.3,0.7]
print("\n Observable layer  matrix:\n",b_df)
b = b_df.values


 Observable layer  matrix:
            Batsman Bowler
Boundary       0.8    0.2
single run     0.6    0.4
wicket         0.3    0.7


# Viterbi Algorithm

The Viterbi algorithm is an algorithm for finding the most likely sequence of hidden states—called the Viterbi path—that results in a sequence of observed events, especially in the context of Markov information sources and hidden Markov models (HMM).The Viterbi algorithm is named after Andrew Viterbi, who proposed it in 1967 

In [52]:
def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    # init blank path
    path = path = np.zeros(T,dtype=int)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))
    
    # init delta and phi 
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    print('\nStart Walk Forward\n')    
    # the forward algorithm extension
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    print('-'*50)
    print('Start Backtrace\n')
    path[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]
        print('path[{}] = {}'.format(t, path[t]))
        
    print("After doing the backtrace it will give us the output as shown below")
        
    return path, delta, phi

In [53]:
a

array([[0.3, 0.5, 0.2],
       [0.6, 0.2, 0.2],
       [0.2, 0.7, 0.1]], dtype=object)

In [54]:
b

array([[0.8, 0.2],
       [0.6, 0.4],
       [0.3, 0.7]], dtype=object)

In [55]:
obs

array([1, 1, 0, 1, 0, 0])

In [56]:
path, delta, phi = viterbi(pi, a, b, obs)
state_map = {0:'Boundary', 1:'single run', 2:'wicket'}
state_path = [state_map[v] for v in path]
pd.DataFrame().assign(Observation=obs_seq).assign(Best_Path=state_path)


Start Walk Forward

s=0 and t=1: phi[0, 1] = 2.0
s=1 and t=1: phi[1, 1] = 2.0
s=2 and t=1: phi[2, 1] = 2.0
s=0 and t=2: phi[0, 2] = 1.0
s=1 and t=2: phi[1, 2] = 1.0
s=2 and t=2: phi[2, 2] = 1.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 0.0
s=2 and t=3: phi[2, 3] = 0.0
s=0 and t=4: phi[0, 4] = 1.0
s=1 and t=4: phi[1, 4] = 2.0
s=2 and t=4: phi[2, 4] = 1.0
s=0 and t=5: phi[0, 5] = 1.0
s=1 and t=5: phi[1, 5] = 0.0
s=2 and t=5: phi[2, 5] = 0.0
--------------------------------------------------
Start Backtrace

path[4] = 0
path[3] = 1
path[2] = 0
path[1] = 1
path[0] = 2
After doing the backtrace it will give us the output as shown below


,Observation,Best_Path
0,Bowler,wicket
1,Bowler,single run
2,Batsman,Boundary
3,Bowler,single run
4,Batsman,Boundary
5,Batsman,single run
